In [1]:
import cv2
import numpy as np
import os

In [2]:
prototxt_path = "deploy.prototxt"
model_path = "res10_300x300_ssd_iter_140000.caffemodel"

In [3]:
model =  cv2.dnn.readNetFromCaffe(prototxt_path,model_path)

In [4]:
import os
directory = 'E:\Instagram\Islamabad Sketch'
os.chdir(directory)

In [5]:
img = cv2.imread('E:/Instagram/Islamabad/05.jpg')

In [6]:
h, w = img.shape[:2]
kernel_width = (w // 7) | 1
kernel_height = (h // 7) | 1
blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))
model.setInput(blob)
output = np.squeeze(model.forward())

for i in range(0, output.shape[0]):
    confidence = output[i, 2]
    if confidence > 0.4:
        box = output[i, 3:7] * np.array([w, h, w, h])
        start_x, start_y, end_x, end_y = box.astype(np.int)
        face = img[start_y: end_y, start_x: end_x]
        face = cv2.GaussianBlur(face, (kernel_width, kernel_height), 0)
        img[start_y: end_y, start_x: end_x] = face

scale = 0.60
width = int(img.shape[1]*scale)
height = int(img.shape[0]*scale)

dim = (width,height)
resized = cv2.resize(img,dim,interpolation = cv2.INTER_AREA)

kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1,9,0],
                              [-1,-1,-1]])
sharpened = cv2.filter2D(resized,-1,kernel_sharpening)
gray = cv2.cvtColor(sharpened , cv2.COLOR_BGR2GRAY)

def sketch(image,mask):
    return  cv2.divide(image,255-mask,scale=256)
pencil_image = sketch(gray,110)


In [7]:
filename = '015.png'
cv2.imwrite(filename, pencil_image) 
print(os.listdir(directory)) 

['01.png', '015.png', '02.png', '03.png', '04.png', '05.png', '06.png', '07.png', '08.png']


In [13]:
cv2.imshow('Original', resized)
cv2.imshow('pencil sketch',pencil_image)
cv2.waitKey(0)
cv2.destroyAllWindows()